# Relatório TP1 - ☯ Coding Dojo ☯

**Alunos:**
* José Antônio Carneiro Ávila
* Gleiston Guimarães de Assis Filho
* Rafael Silvério de Sá Lopes
* Raphael de Assis Silva

## Principais desafios, decisões e arquitetura utilizada

Na realização deste trabalho foi implementado um page fetcher e um escalonador para coletar páginas de diversos domínios na internet. Os principais desafios encontrados estão relacionados à performance da aplicação. A fim de paralelizar os downloads das páginas e aumentar a velocidade da coleta, desenvolveu-se os métodos da classe do page fetcher como assincronos. Dessa forma, pode-se criar vários page fetchers para baixar as páginas dos domínios escolhidos, tornando a busca escalável.

Na arquitetura implementada, a classe escalonador gerencia as páginas visitadas e define quais serão as próximas URLs a serem visitadas. Por outro lado, a classe page fetcher visita cada página selecionada e realiza o download de seu conteúdo. A partir da análise de cada página, coleta-se mais URLs que poderão ser visitadas futuramente, de acordo com a priorização feita pelo escalonador.

## URLs sementes utilizadas

Sementes escolhidas para os testes:
* https://translate.google.com.br/
* https://www.invertexto.com/contador-caracteres
* https://www.americanas.com.br/
* http://www.google.com.br
* https://www.folha.uol.com.br/

Essas páginas foram escolhidas de forma a testar bem o funcionamento do código. As primeiras páginas possuem poucos links para outras páginas, o que é bom para testarmos se o código esta de fato passando por todos os domínios selecionados. As últimas páginas escolhidas possuem muitos links como o site da Folha de São Paulo, o que é bom para testarmos a performance do coletor de links.

## Como foi feito

### Domain

A classe domain representa os domínios da nossa aplicação e possui três atributos principais, são eles:
* int_time_limit_between_requests, que representa o tempo de espera entre as requisições das páginas desse domínio;
* nam_domain, que representa o nome do domínio;
* time_last_access, que representa o horário do último acesso à páginas deste domínio

### Scheduler (Escalonador)

O esquema de armazenamento do coletor é feita por um dicionário e esta ilustrada na imagem abaixo.

<figure style="text-align:center">
    <img src="imgs/estrutura_coletor.png">
    <caption>Fonte: Baeza-Yates e Ribeiro-Neto, 2011 </caption>
</figure>

Na ilustração acima, cada chave do dicionário é um objeto da classe Domain. Os valores do dicionário é uma lista de tuplas com a URL a da página a ser pesquisada e sua profundidade no domínio.

Organizamos as informações dos domínios das páginas coletadas em uma classe chamada scheduler (escalonador). Esta classe é responsável por armazenar as URLs das páginas descobertas e os metadados relacionados à elas, como a profundidade da página em seu domínio principal e se essa página pode ser pesquisada obedecendo as regras do robots.txt. O escalonador também é responsável por controlar o tempo de requisição de páginas em um mesmo domínio, garantindo que não será feito muitas requisições em pouco intervalo de tempo. Por fim, também é responsábilidade do escalonador determinar os critérios de finalização da busca, seja por limite de páginas pesquisadas, por tempo de busca ou por limite de profundidade da coleta.

### Page fecther

O Page fetcher é a classe que visita as URLs dadas pelo escalonador e obtêm as páginas referenciadas por elas. Para atingir tal objetivo analisa-se o conteúdo HTML da página e extrai-se os links contidos nos atributos *href* nas tags *a*. Identificados esses links,enviamos para o escalonador adicionar na lista de páginas a serem pesquisadas.

## Análise de performance

In [ ]:
from crawler.scheduler import *
from crawler.page_fetcher import *
from urllib.parse import urlparse
from datetime import datetime, timedelta

def start_escalonador():
    Escalonador = Scheduler(str_usr_agent="Wall-E",
                                    int_page_limit=10,
                                    int_depth_limit=6,
                                    arr_urls_seeds=["https://www.casasbahia.com.br",
                                                    "https://www.r7.com",
                                                    "https://www.americanas.com.br",
                                                    "http://www.google.com.br",
                                                    "https://www.folha.uol.com.br"])

    Escalonador.add_new_page(urlparse("https://www.casasbahia.com.br"), 1)
    Escalonador.add_new_page(urlparse("https://www.r7.com"), 1)
    Escalonador.add_new_page(urlparse("https://www.americanas.com.br"), 1)
    Escalonador.add_new_page(urlparse("http://www.google.com.br"), 1)
    Escalonador.add_new_page(urlparse("https://www.folha.uol.com.br"), 1)

    return Escalonador

for threads in range(10,100,20):
    Escalonador = start_escalonador()
    tempoInicial = datetime.now()
    fetchers = [PageFetcher(Escalonador) for i in range(threads)]
    print(len(fetchers))
    for f in fetchers:
        f.run()
    tempoFinal = datetime.now()
    print(''*15,'\n\n')
    print('Tempo total:',tempoFinal-tempoInicial,'\n\n\n\n')

10
